## 1.先把csv里的数据读取出来

这里要注意一点就是，需要sys.path.append添加项目根路径，<br>
不然的话，python执行会找不到config和tools这两个自定义<br>
模块，而且需要写在import之前！

In [2]:
import requests as rq
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
import config as c
import tools as t

result_save_file = c.outputs_logs_path + 'wzly.csv'
raw_data = pd.read_csv(result_save_file)

## 2.分析身高情况

这里要对数据做下清洗，流程如下：

- 1.筛选140-200以内的身高值
- 2.把身高按照升序排列
- 3.按照三个身高段，对数据进行分段处理，依次是140,150,160,170,180

In [23]:
from pyecharts import Bar
    
# 身高范围
height_interval = ['140cm', '150cm', '160cm', '170cm', '180cm'] 

# 分析身高
def analysis_height(data):
    height_data = data['身高']
    height = (height_data.loc[(height_data > 140) & (height_data < 200)]).value_counts().sort_index()
    height_count = [0, 0, 0, 0, 0]
    for h in range(0, len(height)):
        if 140 <= height.index[h] < 150:
            height_count[0] += height.values[h]
        elif 150 <= height.index[h] < 160:
            height_count[1] += height.values[h]
        elif 160 <= height.index[h] < 170: 
            height_count[2] += height.values[h]
        elif 170 <= height.index[h] < 180:
            height_count[3] += height.values[h]
        elif 180 <= height.index[h] < 190:
            height_count[4] += height.values[h]
    return height_count

# 绘制身高分布柱状图
def draw_height_bar(data):
    bar = Bar("妹子身高分布柱状图")
    bar.add("妹子身高",height_interval,data,bar_category_gap=0,is_random=True,)
    return bar
    

draw_height_bar(analysis_height(raw_data))

In [24]:
from pyecharts import Pie

# 绘制身高分布饼图
def draw_height_pie(data):
    pie = Pie("妹子身高分布饼图-圆环图",title_pos='center')
    pie.add("", height_interval, data, radius=[40, 75], label_text_color=None,
        is_label_show=True, legend_orient='vertical',is_random=True,
        legend_pos='left')
    return pie

draw_height_pie(analysis_height(raw_data))

**分析时间**：

从上面两个图可以看出，占比最高的身高范围是：**160-170cm**，占比达**68.13%**；<br>
其次是 **150-160cm**，占比 **26.08%**，170-180cm的占比5.61%，(¯﹃¯)啧啧，<br>
像我这种勉强到170cm的应该还是有比较多机会的。

---

## 3.分析学历情况

接着来分析一波学历情况

In [43]:
from pyecharts import Funnel

edu_interval = ['本科', '大专', '高中', '中专', '初中', '硕士', '博士', '院士']  # 学历范围

# 分析学历
def analysis_edu(data):
    return data['学历'].value_counts().values


# 学历漏斗图
def draw_edu_funnel(data):
    funnel = Funnel("妹子学历分布漏斗图",title_top='center')
    funnel.add("学历", edu_interval, data, is_label_show=True,label_pos="inside", label_text_color="#fff",is_random=True )
    return funnel

draw_edu_funnel(analysis_edu(raw_data))

**分析时间**：

从上面的漏斗图看出，交友的大部分妹子大部分都是本科与大专的，高中，中专和初中其次，<br>
最令我好奇的是竟然有4个院士？？？怀着好奇心我搜了下csv里对应的数据：<br>
一个是良缘小秘书，我主良缘官方帐号；<br>接着有两个：<br>
http://www.7799520.com/user/616763.html<br>
http://www.7799520.com/user/554309.html<br>
最后还有一个：<br>
http://www.7799520.com/user/418246.html<br>
打开一看:<br>
![](http://static.zybuluo.com/coder-pig/5czocb1cplikn7achj89xu8d/1.png)
然后官网还写着这样的骚话：
![](http://static.zybuluo.com/coder-pig/d8ijdz40wg5tr6uk4ul26dzr/2.png)
我TM...
![](http://static.zybuluo.com/coder-pig/dtdec6xdj1770it2s4axwdmc/3.png)

---

## 4.分析年龄情况

接着来分析以下都是什么年龄段的妹子找对象(什么年龄段的妹子比较着急...)<br>
吐槽下年龄这个，填资料的人不是一般的皮：<br>

![](http://static.zybuluo.com/coder-pig/2iqvey783t2a923ea4xgcdx6/1.png)

![](http://static.zybuluo.com/coder-pig/3yaev1s475eeo4kwzl61c5oe/2.png)

<br>
这些垃圾值肯定是要过滤一波的，所有筛选的年龄段范围为：1956-2000，<br>
分为五个年龄段：18-25,25-30,31-40,41-50和50以上。<br>
接着写代码过滤下统计一波！<br>

In [59]:
from pyecharts import Radar

# 学历范围
age_interval = [
    ('18-25', 8000), ('26-30', 8000), ('31-40', 8000),
    ('41-50', 8000), ('50以上', 8000),
] 

# 分析年龄
def analysis_age(data):
    age_data = data['出生年份']
    age = (age_data.loc[(age_data >= 1956) & (age_data <= 2000)]).value_counts().sort_index()
    age_count = [0, 0, 0, 0, 0]
    for h in range(0, len(age)):
        if 1993 <= age.index[h] <= 2000:
            age_count[0] += age.values[h]
        elif 1988 <= age.index[h] <= 1992:
            age_count[1] += age.values[h]
        elif 1978 <= age.index[h] <= 1987:
            age_count[2] += age.values[h]
        elif 1968 <= age.index[h] <= 1977:
            age_count[3] += age.values[h]
        elif age.index[h] < 1968:
            age_count[4] += age.values[h]
    return age_count

# 年龄雷达图
def draw_age_radar(data):
    radar = Radar("妹子年龄分布雷达图")
    radar.config(age_interval)
    radar.add("年龄段", data, is_area_show=False,legend_selectedmode='single')
    return radar

draw_age_radar([analysis_age(raw_data)])

**分析时间**：

从雷达图上面可以看出，26-30比较热衷与找对象脱单，人数达到了**7355人**，<br>
接着是18-25的，有**4313人**之多，正常，女的都结婚比较早，想想小学同学<br>
的儿子都上小学了，我还单着，**哭哭/(ㄒoㄒ)/~~ **。接着是31-40岁的大龄剩女，<br>
也有3417人那么多，都是到了年纪才着急么，恐怖...50岁以上的则有77人。

---

## 5.分析下城市

